In [15]:
import pandas as pd
a = pd.read_excel("/Users/MikeImac/Desktop/FinaceTool/经营检测表（费用明细表）数据底稿.xlsx",sheet_name="之了成都教育2206")

In [16]:
a

,科目代码,科目名称/部门,项目,店铺/HS编码,HS编码,币别,期初借方余额,期初贷方余额,本期借方发生额,本期贷方发生额,本年借方累计,本年贷方累计,期末借方余额,期末贷方余额
0,1002,银行存款,NaN,NaN,NaN,NaN,3030.00,0.00,3030.67,6200.00,3992.67,4197.33,6120.00,0.00
1,1002.02,"民生银行（634342543,634770624）",NaN,NaN,NaN,NaN,1248.00,0.00,6193.33,1668.00,2904.00,2466.00,5499.33,0.00
2,1002.02,001之了（成都）教育科技有限公司,NaN,NaN,NaN,NaN,4888.67,0.00,5367.33,5162.00,4866.00,764.00,4804.67,0.00
3,1012,其他货币资金,NaN,NaN,NaN,NaN,2178.67,0.00,2404.00,5661.33,5959.33,2508.00,3628.00,0.00
4,1012.03,支付宝-备付金账户（2070664869@qq.com）,NaN,NaN,NaN,NaN,0.00,0.00,1872.00,4136.67,3976.67,2181.33,2772.67,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,6711.05,论文项目,排版,001之了（成都）教育科技有限公司,000-00,NaN,0.00,0.00,3117.33,2324.00,3980.67,6452.67,0.00,0.00
215,6711.06,非常损失,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,3536.67,3998.67,0.00,0.00
216,6711.06,总裁办,综合,001之了（成都）教育科技有限公司,0,NaN,0.00,0.00,0.00,0.00,6532.00,3654.67,0.00,0.00
217,6711.06,Python项目组,编程,001之了（成都）教育科技有限公司,0,NaN,0.00,0.00,0.00,0.00,5272.67,6284.00,0.00,0.00


In [20]:
a.index

RangeIndex(start=0, stop=219, step=1)